In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from signals import *
from frequencyestimator import *
from scipy.optimize import basinhopping, minimize
from tqdm.auto import tqdm

sns.set_style("whitegrid")
sns.despine(left=True, bottom=True)
sns.set_context("poster", font_scale = .45, rc={"grid.linewidth": 0.8})

/Users/farrokhlabib/Documents/github/csAE/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<Figure size 640x480 with 0 Axes>

In [2]:
P0 = lambda n, theta: np.cos((2*n+1)*theta)**2
P1 = lambda n, theta: np.sin((2*n+1)*theta)**2

def estimate_signal(depths, n_samples, theta):
        signals = np.zeros(len(depths), dtype = np.complex128)
        measurements = np.zeros(len(depths))
        for i,n in enumerate(depths):
            # Get the exact measuremnt probabilities
            p0 = P0(n, theta)
            p1 = P1(n, theta)

            p0x = P0x(n,theta)
            p1x = P1x(n,theta)
            
            p0_estimate = np.random.binomial(n_samples[i], p0)/n_samples[i]
            p1_estimate = 1 - p0_estimate

            p0x_estimate = np.random.binomial(n_samples[i], p0x)/n_samples[i]
            p1x_estimate = 1.0 - p0x_estimate
            measurements[i] = p0_estimate
            
            # Estimate theta
            theta_estimated = np.arctan2(p0x_estimate - p1x_estimate, p0_estimate - p1_estimate)

            # Compute f(n) - Eq. 3
            fi_estimate = np.exp(1.0j*theta_estimated)
            signals[i] = fi_estimate
         
        return signals, measurements

from scipy.stats import binom

def apply_correction(ula_signal, measurements, theta_est, theta):
    p_exact = np.cos((2*ula_signal.depths+1)*(theta))**2
    p_neg = np.cos((2*ula_signal.depths+1)*(-theta))**2
    p_o2 = np.cos((2 * ula_signal.depths + 1) * (theta_est/2.0)) ** 2
    p_o4 = np.cos((2 * ula_signal.depths + 1) * (theta_est/4.0)) ** 2
    p_same = np.cos((2*ula_signal.depths+1)*(theta_est))**2
    p_s2 = np.cos((2 * ula_signal.depths + 1) * (np.pi/2-theta_est)) ** 2
    p_s4 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 4 - theta_est)) ** 2
    p_s2_o2 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 2 - theta_est/2)) ** 2
    p_s4_o2 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 4 - theta_est/2)) ** 2

    l_exact = np.sum(
        np.log([1e-75 + binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk],
                                  p_exact[kk]) for kk in
                range(len(ula_signal.n_samples))]))
    l_neg = np.sum(
        np.log([1e-75 + binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk],
                                  p_neg[kk]) for kk in
                range(len(ula_signal.n_samples))]))
    l_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk]*measurements[kk], ula_signal.n_samples[kk], p_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_o4 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_o4[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_same = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_same[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s4 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s4[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s2_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s2_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s4_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s4_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    
    which_correction = np.argmax([l_same, l_s2, l_s4, l_o2, l_o4, l_s2_o2, l_s4_o2, l_neg])
    if which_correction == 1:
        theta_est = np.pi/2.0 - theta_est
    elif which_correction == 2:
        theta_est = np.pi/4.0 - theta_est
    elif which_correction == 3:
        theta_est = theta_est/2
    elif which_correction == 4:
        theta_est = theta_est/4
    elif which_correction == 5:
        theta_est = np.pi / 2.0 - 0.5 * theta_est
    elif which_correction == 6:
        theta_est = np.pi / 4.0 - 0.5 * theta_est
    # elif which_correction == 7:
    #     theta_est = -theta_est

    # print(f'FINAL ANGLE FOUND: {theta_est, theta}')

    return theta_est

In [3]:
import numpy as np
import itertools
from typing import List

def generate_all_sign_variations(signs_array: np.ndarray) -> np.ndarray:
    """
    Generate all possible sign variations by changing signs at all possible pairs of positions.
    
    Parameters:
    -----------
    signs_array : numpy.ndarray
        Original array of signs (typically containing 1 or -1)
    
    Returns:
    --------
    numpy.ndarray
        Array of all possible sign variation arrays
    """
    # Get all possible unique pairs of positions
    n = len(signs_array)
    position_pairs = list(itertools.combinations(range(n), 2))
    
    # Will store all variations
    all_variations = []
    
    # Iterate through all possible position pairs
    for pos1, pos2 in position_pairs:
        # Generate variations for this pair of positions
        pair_variations = generate_pair_variations(signs_array, pos1, pos2)
        all_variations.extend(pair_variations)
    
    return all_variations

def generate_adjacent_sign_variations(signs_array: np.ndarray) -> np.ndarray:
    """
    Generate sign variations by sliding a two-position window across the array.
    
    Parameters:
    -----------
    signs_array : numpy.ndarray
        Original array of signs (typically containing 1 or -1)
    
    Returns:
    --------
    numpy.ndarray
        Array of all possible sign variation arrays
    """
    # Total length of the array
    n = len(signs_array)
    
    # Will store all variations
    all_variations = []
    
    # Iterate through adjacent position pairs 
    # (0,1), (1,2), (2,3), ... until the second-to-last pair
    for pos1 in range(1, n - 1):
        pos2 = pos1 + 1
        
        # Generate variations for this pair of adjacent positions
        pair_variations = generate_pair_variations(signs_array, pos1, pos2)
        all_variations.extend(pair_variations)
    
    return all_variations

def generate_pair_variations(signs_array: np.ndarray, pos1: int, pos2: int) -> List[np.ndarray]:
    """
    Generate sign variations for a specific pair of positions.
    
    Parameters:
    -----------
    signs_array : numpy.ndarray
        Original array of signs
    pos1 : int
        First position to modify
    pos2 : int
        Second position to modify
    
    Returns:
    --------
    List[numpy.ndarray]
        List of sign variation arrays
    """
    # Validate input positions
    if pos1 < 0 or pos2 < 0 or pos1 >= len(signs_array) or pos2 >= len(signs_array):
        raise ValueError("Positions must be within the array bounds")
    
    # Generate all possible sign combinations for the two positions
    sign_combinations = list(itertools.product([-1, 1], repeat=2))
    
    # Create variations
    variations = []
    for combo in sign_combinations:
        # Create a copy of the original array
        variation = signs_array.copy()
        
        # Modify the two specified positions
        variation[pos1] *= combo[0]
        variation[pos2] *= combo[1]
        
        variations.append(tuple(variation))
    
    return variations


# Example array
signs = np.array([1]*12)

# Generate variations for all position pairs
all_variations = generate_adjacent_sign_variations(signs)

print("Original array:", signs)
print(f"\nTotal variations: {len(all_variations)}")
print(f'4*len(signs)^2: {4*len(signs)**2}')
print(f'2^len(signs): {2**len(signs)}')

print("\nFirst few variations:")
for var in all_variations[:10]:  # Print first 10 variations
    print(var)

Original array: [1 1 1 1 1 1 1 1 1 1 1 1]

Total variations: 40
4*len(signs)^2: 576
2^len(signs): 4096

First few variations:
(1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
(1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
(1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
(1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1)
(1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
(1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1)
(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
(1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1)
(1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1)


## Define the objective function to minimize

In [4]:
def objective_function(lp, cos_signal, abs_sin, ula_signal, esprit):   
    signal = cos_signal + 1.0j * lp * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    _, _ = esprit.estimate_theta_toeplitz(R)
    eigs = np.abs(esprit.eigs)[:2]
    obj = eigs[1] - eigs[0]

    return obj

## What is the distribution of the signs? 

In [12]:
narray = [2,2,2,2,2,2,2,3]

# avals = np.linspace(0.1, 0.9, 8)
avals = [0.1*i for i in range(1, 10)]

ula_signal = TwoqULASignal(M=narray, C=5)
esprit = ESPIRIT()

num_mc = 1000

sign_distributions = {}

for a in avals:
    theta = np.arcsin(a)
    distr = {}
    for i in range(num_mc):
        signal, _ = estimate_signal(ula_signal.depths, ula_signal.n_samples, theta)
        # print(measurements)
        signs = tuple(np.sign(np.imag(signal)))
        # print(signs)

        distr[signs] = distr.get(signs, 0.0) + 1/num_mc
    sign_distributions[str(a)] = distr



In [13]:
2**len(ula_signal.depths), len(sign_distributions['0.1'])

(1024, 17)

In [14]:
all_signs = set()
for a in avals:
    distr = sign_distributions[str(a)]
    for sign in distr.keys():
        all_signs.add(sign)
len(all_signs)

124

In [15]:
len(all_signs)/2**len(ula_signal.depths)

0.12109375

In [16]:
narray = [2,2,2,2,2,3]

def get_heavy_signs(narray, steps):

    avals = np.linspace(0.1, 0.9, steps)

    ula_signal = TwoqULASignal(M=narray, C=5)

    num_mc = 1000

    sign_distributions = {}

    for a in avals:
        theta = np.arcsin(a)
        distr = {}
        for i in range(num_mc):
            signal, _ = estimate_signal(ula_signal.depths, ula_signal.n_samples, theta)
                # print(measurements)
            signs = tuple(np.sign(np.imag(signal)))
            # print(signs)

            distr[signs] = distr.get(signs, 0.0) + 1/num_mc
        sign_distributions[str(a)] = distr

    def get_signs(sign_distribution):
        # Sort the dictionary by values in descending order
        sorted_data = dict(sorted(sign_distribution.items(), key=lambda item: item[1], reverse=True))

        # Create a new dictionary with cumulative sum close to 0.68
        cumulative_dict = {}
        cumulative_sum = 0.0

        for key, value in sorted_data.items():
            cumulative_dict[key] = value
            if cumulative_sum + value > 0.68:
                break
            cumulative_sum += value

        # Output the result
        return cumulative_dict
    
    ret_dict = {}
    for a in avals:
        distr = get_signs(sign_distributions[str(a)])
        ret_dict[str(a)] = distr
 
    # Normalize the values
    normalized_data = {}
    for key, sub_dict in ret_dict.items():
        total_sum = sum(sub_dict.values())
        normalized_data[key] = {k: v / total_sum for k, v in sub_dict.items()}

    return normalized_data

normalized_data = get_heavy_signs(narray, 10)


In [17]:
import ast
# Taking a random sample based on the normalized probabilities
def sample_from_normalized(data, sample_size=1):
    keys = list(map(str, list(data.keys())))
    probabilities = list(data.values())
    sampled_keys = np.random.choice(a=keys, size=sample_size, p=probabilities)
    return [ast.literal_eval(t) for t in sampled_keys]
avals = list(normalized_data.keys())
# Get one random sample
sampled = sample_from_normalized(normalized_data[avals[0]], sample_size=3)
print("Sampled keys:",sampled)

Sampled keys: [(1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0), (1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0), (1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0)]


In [18]:
aa_milne_arr = ['pooh', 'rabbit', 'piglet', 'Christopher']
np.random.choice(aa_milne_arr, 5, p=[0.5, 0.1, 0.1, 0.3])

array(['pooh', 'pooh', 'pooh', 'Christopher', 'rabbit'], dtype='<U11')

## Use the distribution of signs so we don't have to try all possible signs

In [75]:
# np.random.seed(42)

a = 0.9
print(a)
theta = np.arccos(a)

narray = [2,2,2,2,2,2,2,2]
heavy_signs = get_heavy_signs(narray, 2*len(narray))

ula_signal = TwoqULASignal(M=narray, C=5)
esprit = ESPIRIT()

def csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=False, correction=False, optimize=False, method='nelder-mead', maxiter=10, disp=False):

    depths = ula_signal.depths
    n_samples = ula_signal.n_samples
    csignal, measurements = estimate_signal(depths, n_samples, theta)

    cos_signal = np.real(csignal)

    correct_signs = np.sign(np.imag(csignal))
    if disp:
        print(f'correct signs: {correct_signs}')
    abs_sin = np.abs(np.imag(csignal))

    bounds=[[-1,1] for _ in range(len(depths))]

    avg_nfev = 0.0
    obj = 1.0
    if sample:
        avals = list(heavy_signs.keys())
        signs_to_try = {}
        for a in avals:
            signs_to_try[a] = list(set(sample_from_normalized(heavy_signs[a], sample_size=2)))

        if optimize:
            # rough estimate where the amplitude is
            a0 = np.sqrt(0.5 + 0.5 * cos_signal[0])
            # use rough estimate to determine which set of signs to look at
            avals = list(map(float, avals))

            # find first element in avals that is larger than a0
            left = 0
            right = len(avals)
            
            while left < right:
                mid = (left + right) // 2
                
                if avals[mid] <= a0:
                    left = mid + 1
                else:
                    right = mid
            
            if left >= 3:
                avals_to_use = list(map(str, avals[left-3: left+3]))
            elif left>=2:
                avals_to_use = list(map(str, avals[left-2: left+3]))
            elif left>=1:
                avals_to_use = list(map(str, avals[left-1: left+3]))
            else:
                avals_to_use = list(map(str, avals[left: left+3]))

            if disp:
                print(f'rough estimate a: {a0}')
                print(f'avals: {avals}')
                print(f'avals to use: {avals_to_use}')

            all_signs = []
            for a in avals_to_use:
                for x in signs_to_try[a]:
                    hamming_distance_two_signs = generate_adjacent_sign_variations(np.array(x))
                    all_signs.extend(hamming_distance_two_signs)
            all_signs = list(set(all_signs))
            if disp:
                print(f'number of signs Hamming distance two: {len(all_signs)}')

            for x in all_signs:
                curr_obj = objective_function(np.array(x), cos_signal, abs_sin, ula_signal, esprit)
                if curr_obj<obj:
                    if disp:
                        print(f'current objective: {curr_obj}')
                        print(f'current best signs: {x}')
                    obj = curr_obj
                    x_star = np.array(x)


        else:
            for x in signs_to_try:
                basin_obj = objective_function(np.array(x), cos_signal, abs_sin, ula_signal, esprit)
                if basin_obj < obj:
                    obj = basin_obj
                    x_star = np.array(x)
    else:
        signs_to_try = []
        for a in heavy_signs.keys():
            signs = heavy_signs[a].keys()
            signs_to_try += signs
        signs_to_try = list(set(signs_to_try))
        if disp:
            print(f'number of signs to try: {len(signs_to_try)}')
        for x in signs_to_try:
            if optimize:
                res = minimize(objective_function, x, args=(cos_signal, abs_sin, ula_signal, esprit), bounds=bounds, method=method, options={'maxiter': maxiter, 'disp': False})
                avg_nfev += res.nfev
                if disp:
                    print(res.x)
                if disp:
                    print(f'number fev: {res.nfev}')
                if res.fun < obj:
                    obj = res.fun
                    x_star = np.sign(res.x)
            else:
                basin_obj = objective_function(np.array(x), cos_signal, abs_sin, ula_signal, esprit)
                if basin_obj < obj:
                    obj = basin_obj
                    x_star = np.array(x)
    if disp:
        print(x_star)
        
    signal = cos_signal + 1.0j * x_star * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    theta_est, _ = esprit.estimate_theta_toeplitz(R)
    eigs = np.abs(esprit.eigs)[:2]
    basin_obj = eigs[1] - eigs[0]
    if correction:
        theta_est = apply_correction(ula_signal, measurements, theta_est, theta) #apply correction
    if disp:
        print(f'basin obj: {basin_obj}')

    cR = ula_signal.get_cov_matrix_toeplitz(csignal)
    theta_est1, _ = esprit.estimate_theta_toeplitz(cR)
    eigs = np.abs(esprit.eigs)[:2]
    true_obj = eigs[1] - eigs[0]
    if disp:
        print(f'true obj: {true_obj}')

    num_queries = np.sum(np.array(ula_signal.depths)*np.array(ula_signal.n_samples)) + ula_signal.n_samples[0]
    # Compute the maximum single query
    max_single_query = np.max(ula_signal.depths)

    ret_dict = {'theta_est': theta_est, 'theta_est1': theta_est1, 
                'error': np.abs(np.sin(theta)-np.sin(theta_est)), 
                'error1': np.abs(np.sin(theta)-np.sin(theta_est1)), 
                'queries': num_queries, 'depth': max_single_query,
                'true_obj': true_obj, 'basin_obj': basin_obj,
                'x_star': x_star, 'avg_nfev': avg_nfev/len(signs_to_try)}
    
    return ret_dict

csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=True, correction=True, optimize=True, disp=True)

0.9
correct signs: [ 1.  1. -1.  1.  1. -1.  1. -1. -1.]
rough estimate a: 0.8845009654013923
avals: [0.1, 0.15333333333333335, 0.20666666666666667, 0.26, 0.31333333333333335, 0.3666666666666667, 0.42000000000000004, 0.4733333333333334, 0.5266666666666667, 0.5800000000000001, 0.6333333333333333, 0.6866666666666666, 0.74, 0.7933333333333333, 0.8466666666666667, 0.9]
avals to use: ['0.74', '0.7933333333333333', '0.8466666666666667', '0.9']
number of signs Hamming distance two: 64
current objective: -46.70756103355299
current best signs: (1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0)
current objective: -87.51902743281335
current best signs: (1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0)
current objective: -130.44095251902027
current best signs: (1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0)
current objective: -165.1270803775468
current best signs: (1.0, -1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0)
current objective: -262.35457484431583
current best signs: (1.0, -1.0, -1.0, 1.0, 1.0, 

{'theta_est': 0.45012265741876556,
 'theta_est1': 0.4501226574187656,
 'error': 0.0008139169977984251,
 'error1': 0.0008139169977983696,
 'queries': 2600,
 'depth': 128,
 'true_obj': -373.8884219968161,
 'basin_obj': -373.88842199681574,
 'x_star': array([ 1.,  1., -1.,  1.,  1., -1.,  1., -1., -1.]),
 'avg_nfev': 0.0}

## Let's do some statistics now

In [20]:
np.random.seed(42)

avals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# avals = [0.2]
# avals = [0.5, 0.6, 0.7, 0.8, 0.9]
narray = [2]*6
heavy_signs = get_heavy_signs(narray, len(narray))
num_queries = np.zeros(len(avals), dtype=int)
max_single_query = np.zeros(len(avals), dtype=int)

num_mc=100
thetas = np.zeros((len(avals), num_mc))
errors = np.zeros((len(avals), num_mc))
thetas1 = np.zeros((len(avals), num_mc))
errors1 = np.zeros((len(avals), num_mc))

basin_obj = np.zeros((len(avals), num_mc))
true_obj = np.zeros((len(avals), num_mc))

ula_signal = TwoqULASignal(M=narray, C=3)
esprit = ESPIRIT()

for j,a in enumerate(avals):
    theta = np.arcsin(a)
    print(f'theta = {theta}')
    for i in tqdm(range(num_mc)):
        res = csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=False, optimize=False, disp=False)
        thetas[j][i] = res['theta_est']
        errors[j][i] = res['error']

        thetas1[j][i] = res['theta_est1']
        errors1[j][i] = res['error1']

        basin_obj[j][i] = res['basin_obj']
        true_obj[j][i] = res['true_obj']
    num_queries[j] = res['queries']
    max_single_query[j] = res['depth']
    avg_nfev = res['avg_nfev']
    
    print(f'average nfev: {avg_nfev}')

    print(f'constant factors query and depth: {np.percentile(errors[j], 95) * num_queries[j]}, {np.percentile(errors[j], 95) * max_single_query[j]}')




theta = 0.1001674211615598


100%|██████████| 100/100 [00:02<00:00, 35.22it/s]


average nfev: 0.0
constant factors query and depth: 359.56702856489574, 28.62225103004145
theta = 0.2013579207903308


100%|██████████| 100/100 [00:02<00:00, 34.96it/s]


average nfev: 0.0
constant factors query and depth: 313.1624498813814, 24.928354219413446
theta = 0.30469265401539747


...

average nfev: 0.0
constant factors query and depth: 245.46375004834394, 19.5394029889229
theta = 0.9272952180016123


100%|██████████| 100/100 [00:02<00:00, 35.36it/s]


average nfev: 0.0
constant factors query and depth: 42.030970171840025, 3.345748869400201
theta = 1.1197695149986342


100%|██████████| 100/100 [00:02<00:00, 34.68it/s]

average nfev: 0.0
constant factors query and depth: 2.0137509499497646, 0.16029858308057826


In [21]:
np.array(basin_obj[0])/np.array(true_obj[2]), np.sum(np.where(np.array(basin_obj[2])/np.array(true_obj[2]) >= 0.99, 1, 0))/len(basin_obj[2])

(array([0.86466878, 0.92203677, 1.43514318, 0.74914088, 0.80513099,
        1.17173609, 1.00854589, 1.08820042, 1.24863459, 1.41155735,
        1.09163622, 0.99051616, 0.97313793, 1.20272384, 1.4826406 ,
        1.0784456 , 2.60465948, 1.24500867, 1.12362081, 1.10509897,
        1.01923463, 1.05584381, 1.12431391, 0.89759333, 2.48267314,
        1.05733855, 1.36605721, 1.22832311, 0.94459644, 0.91096599,
        1.12084889, 0.64088204, 1.32859372, 0.90285891, 1.10411302,
        1.35947613, 0.91808047, 0.94892788, 1.38528649, 1.27097455,
        1.14698224, 1.01393638, 1.51062667, 0.96299043, 0.87625116,
        1.13065631, 1.16023398, 1.13350927, 0.97442367, 0.93424272,
        1.14796328, 1.70720683, 1.0997591 , 1.1078289 , 1.36765618,
        1.14898905, 1.1663119 , 1.03029201, 1.39274032, 0.97091125,
        1.15029147, 1.14186898, 1.17005952, 1.11067512, 1.00502682,
        0.99667382, 0.8937064 , 0.97827547, 1.42693643, 1.01089424,
        1.08633675, 1.45292614, 1.2294687 , 1.18

In [22]:
for i in range(len(avals)):
    print(np.sum(np.where(np.array(basin_obj[i])/np.array(true_obj[i]) >= 0.99, 1, 0))/len(basin_obj[i]))

0.99
0.11
0.15
0.01
0.0
0.02
0.04
0.04
1.0


In [76]:
np.random.seed(42)

avals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# avals = [np.random.uniform(0.1, 0.9)]
# avals = [0.5, 0.6, 0.7, 0.8, 0.9]
narray = [2]*6
heavy_signs = get_heavy_signs(narray, 2*len(narray))
num_queries = np.zeros(len(avals), dtype=int)
max_single_query = np.zeros(len(avals), dtype=int)

num_mc=100
thetas = np.zeros((len(avals), num_mc))
errors = np.zeros((len(avals), num_mc))
thetas1 = np.zeros((len(avals), num_mc))
errors1 = np.zeros((len(avals), num_mc))

basin_obj = np.zeros((len(avals), num_mc))
true_obj = np.zeros((len(avals), num_mc))

ula_signal = TwoqULASignal(M=narray, C=5)
esprit = ESPIRIT()

for j,a in enumerate(avals):
    theta = np.arccos(a)
    print(f'theta = {theta}')
    for i in tqdm(range(num_mc)):
        res = csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=True, correction=True, optimize=True, disp=False)
        thetas[j][i] = res['theta_est']
        errors[j][i] = res['error']

        thetas1[j][i] = res['theta_est1']
        errors1[j][i] = res['error1']

        basin_obj[j][i] = res['basin_obj']
        true_obj[j][i] = res['true_obj']
    num_queries[j] = res['queries']
    max_single_query[j] = res['depth']
    avg_nfev = res['avg_nfev']
    
    print(f'average nfev: {avg_nfev}')

    print(f'constant factors query and depth: {np.percentile(errors[j], 95) * num_queries[j]}, {np.percentile(errors[j], 95) * max_single_query[j]}')

theta = 1.4706289056333368


100%|██████████| 100/100 [00:08<00:00, 11.74it/s]


average nfev: 0.0
constant factors query and depth: 0.7129912350817033, 0.03405331272032015
theta = 1.3694384060045657


100%|██████████| 100/100 [00:09<00:00, 10.15it/s]


average nfev: 0.0
constant factors query and depth: 2.7692479159892307, 0.13226258703232147
theta = 1.266103672779499


...

average nfev: 0.0
constant factors query and depth: 3.836421166689683, 0.18323205572249232
theta = 0.6435011087932843


100%|██████████| 100/100 [00:10<00:00,  9.21it/s]


average nfev: 0.0
constant factors query and depth: 3.2933203564940863, 0.1572929125489713
theta = 0.45102681179626236


100%|██████████| 100/100 [00:09<00:00, 10.98it/s]

average nfev: 0.0
constant factors query and depth: 10.023773710824187, 0.47874740111399106


In [77]:
np.array(basin_obj[0])/np.array(true_obj[0]), np.sum(np.where(np.array(basin_obj[0])/np.array(true_obj[0]) >= 0.99, 1, 0))/len(basin_obj[0])

(array([1.        , 1.10314115, 1.13779073, 1.        , 1.01134321,
        1.        , 1.130432  , 1.        , 1.        , 1.03232667,
        1.        , 1.0167907 , 1.67681783, 1.        , 1.77114453,
        1.16931639, 1.        , 1.03344381, 1.        , 1.        ,
        1.00890876, 1.08423953, 1.08998822, 1.        , 1.10040985,
        1.37195127, 1.06780749, 1.07448172, 1.10733573, 1.        ,
        1.        , 1.02312906, 1.        , 1.1380952 , 1.        ,
        1.        , 1.        , 1.02173083, 1.08313034, 1.02759955,
        1.07091721, 1.0627728 , 1.        , 1.1349618 , 1.03475596,
        1.        , 1.06733204, 1.        , 1.        , 1.        ,
        1.        , 1.12636008, 1.1820873 , 1.        , 1.05178551,
        1.        , 1.2333474 , 1.15302661, 1.        , 1.        ,
        1.        , 1.0765917 , 1.03380515, 1.14945123, 1.17126228,
        1.46150858, 1.11291827, 1.01473066, 1.        , 1.05029699,
        1.        , 1.20720004, 1.        , 1.  

In [78]:
for i in range(len(avals)):
    print(np.sum(np.where(np.array(basin_obj[i])/np.array(true_obj[i]) >= 0.99, 1, 0))/len(basin_obj[i]))

1.0
1.0
1.0
1.0
1.0
0.92
1.0
0.91
1.0


In [79]:
for i in range(len(avals)):
    print(f'constant factors query and depth (68%): {np.percentile(errors[i], 68) * num_queries[i]}, {np.percentile(errors[i], 68) * max_single_query[i]}')

constant factors query and depth (68%): 0.2544813383491588, 0.012154332577870273
constant factors query and depth (68%): 0.38780094277229454, 0.018521836072706604
constant factors query and depth (68%): 0.6911269588903318, 0.03300904878282182
constant factors query and depth (68%): 0.9208922506828464, 0.04398291346544938
constant factors query and depth (68%): 1.0963233896631956, 0.05236171413316755
constant factors query and depth (68%): 1.2728840292935073, 0.06079446110058542
constant factors query and depth (68%): 2.0534362387714906, 0.09807456662789209
constant factors query and depth (68%): 1.7619830423409157, 0.0841544139625512
constant factors query and depth (68%): 1.6203438700587358, 0.07738955797295455


In [ ]:
Q = 7
lengths = []
for q in range(2, Q + 1):
    narray = [2]*(2*q)
    heavy_signs = get_heavy_signs(narray, 4*len(narray))
    signs_to_try = []
    for a in heavy_signs.keys():
        signs = heavy_signs[a].keys()
        signs_to_try += signs
    signs_to_try = list(set(signs_to_try))
    lengths.append(len(signs_to_try))

lengths

[14, 44, 58, 92, 219, 173]